# Проект 3 (финальный). Создание рекомендательной системы

### Задание: 
У продакт-менеджера есть идея организовать допродажу в корзине для увеличения среднего чека. Для этого нужна рекомендательная система по курсам. Необходимо составить для них таблицу, где каждому ID курса будут предоставлены ещё два курса, которые будут рекомендоваться. 

### Импорт библиотек:

In [1]:
import pandas as pd 
import psycopg2
import psycopg2.extras 
import numpy as np
from  itertools import combinations

### Получение данных

Выгружаем данные с помощью SQL-запроса из базы данных skillfactory:

In [2]:
def getEventsData():
    query = '''with a as
(select 
    c.user_id,
    ci.resource_id,
    c.purchased_at
from final.carts c 
join final.cart_items ci 
        on c.id = ci.cart_id
where ci.resource_type = 'Course' 
    and c.state = 'successful'),

b as
(select 
    c.user_id,
    count(ci.resource_id) cnt
from final.carts c 
join final.cart_items ci 
        on c.id = ci.cart_id
where ci.resource_type = 'Course' 
    and c.state = 'successful'
group by c.user_id)

SELECT
    *
FROM 
    a join b on a.user_id = b.user_id
order by a.user_id'''
    conn = psycopg2.connect("dbname='' user='' host='' password='' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

Посмотрим наши данные:

In [3]:
data = getEventsData()
print(data[:10])

[{'user_id': 51, 'resource_id': 516, 'purchased_at': datetime.datetime(2017, 1, 6, 21, 31, 53, 507000), 'cnt': 2}, {'user_id': 51, 'resource_id': 1099, 'purchased_at': datetime.datetime(2018, 6, 22, 17, 20, 49, 80000), 'cnt': 2}, {'user_id': 974, 'resource_id': 356, 'purchased_at': datetime.datetime(2018, 8, 1, 13, 43, 48, 380000), 'cnt': 1}, {'user_id': 2233, 'resource_id': 1125, 'purchased_at': datetime.datetime(2018, 9, 4, 7, 56, 37, 928000), 'cnt': 1}, {'user_id': 2581, 'resource_id': 1187, 'purchased_at': datetime.datetime(2018, 12, 6, 10, 11, 26, 71000), 'cnt': 1}, {'user_id': 3480, 'resource_id': 745, 'purchased_at': datetime.datetime(2018, 2, 18, 18, 50, 12, 646000), 'cnt': 1}, {'user_id': 4430, 'resource_id': 679, 'purchased_at': datetime.datetime(2017, 11, 16, 6, 27, 54, 482000), 'cnt': 1}, {'user_id': 6117, 'resource_id': 1125, 'purchased_at': datetime.datetime(2018, 8, 1, 5, 1, 45, 31000), 'cnt': 3}, {'user_id': 6117, 'resource_id': 356, 'purchased_at': datetime.datetime(20

Мы получили набор коллекций, в котором присутствует информация по всем покупкам курсов по каждому клиенту: данные по клиентам дублируются и также имеются сведения о покупках пользователями одного курса. Нам необходимо получить информацию только по клиентам, купившим больше одного курса, разбить все покупки курсов на пары, после чего ранжировать их для каждого курса:

In [4]:
user_id = data[0]['user_id']
courses = []
courses_dict = {}

for d in data:
    if user_id != d['user_id']:
        for pair in combinations(set(courses), 2):
            sorted_pair = tuple(sorted(pair))
            if sorted_pair in courses_dict:
                courses_dict[sorted_pair] += 1
            else: 
                courses_dict[sorted_pair] = 1
        user_id = d['user_id']
        courses = []
        courses.append(d['resource_id'])
    else:
        courses.append(d['resource_id'])
        
print(courses_dict)

{(516, 1099): 25, (356, 357): 100, (357, 1125): 52, (356, 1125): 44, (553, 1147): 16, (361, 1138): 40, (1125, 1140): 1, (551, 745): 138, (553, 745): 212, (551, 1138): 14, (553, 568): 83, (514, 517): 10, (514, 566): 138, (517, 566): 21, (562, 563): 53, (363, 563): 33, (511, 563): 19, (363, 562): 77, (511, 562): 55, (363, 511): 99, (568, 745): 102, (509, 745): 59, (509, 553): 48, (1125, 1144): 22, (509, 568): 46, (568, 672): 4, (509, 672): 5, (516, 552): 12, (356, 552): 7, (357, 571): 112, (516, 568): 54, (509, 516): 35, (513, 1141): 34, (571, 1125): 122, (551, 552): 177, (551, 744): 16, (551, 862): 8, (552, 744): 8, (552, 1138): 4, (552, 862): 6, (744, 1138): 1, (744, 862): 2, (862, 1138): 3, (356, 679): 8, (745, 1099): 53, (571, 745): 22, (571, 1099): 2, (568, 1099): 53, (509, 1099): 31, (517, 750): 34, (800, 1125): 4, (569, 840): 204, (745, 1125): 15, (514, 745): 38, (509, 514): 8, (514, 551): 200, (509, 551): 15, (571, 765): 83, (745, 1187): 32, (363, 566): 20, (513, 1125): 1, (1100,

Определим **сколько различных пар курсов** встречаются **вместе в покупках** клиентов:

In [5]:
len(courses_dict)

3989

Найдем **самую популярную** пару курсов:

In [6]:
courses_dict = {k: v for k, v in sorted(courses_dict.items(), key=lambda item: item[1], reverse=True)} # отсортируем данные
courses_dict

{(551, 566): 797,
 (515, 551): 417,
 (489, 551): 311,
 (523, 551): 304,
 (566, 794): 290,
 (489, 515): 286,
 (490, 566): 253,
 (490, 551): 247,
 (570, 752): 247,
 (569, 572): 216,
 (515, 523): 213,
 (553, 745): 212,
 (489, 523): 206,
 (569, 840): 204,
 (514, 551): 200,
 (516, 745): 199,
 (515, 566): 195,
 (489, 566): 188,
 (504, 572): 184,
 (572, 840): 178,
 (551, 552): 177,
 (507, 570): 172,
 (490, 809): 163,
 (489, 490): 152,
 (507, 752): 150,
 (523, 552): 144,
 (490, 515): 143,
 (551, 570): 142,
 (504, 569): 139,
 (514, 515): 139,
 (551, 745): 138,
 (514, 566): 138,
 (502, 551): 135,
 (504, 840): 135,
 (571, 1125): 122,
 (523, 566): 120,
 (502, 566): 120,
 (570, 809): 119,
 (752, 809): 115,
 (490, 523): 114,
 (357, 571): 112,
 (523, 564): 110,
 (551, 749): 109,
 (516, 553): 107,
 (551, 777): 107,
 (551, 679): 104,
 (356, 571): 103,
 (551, 564): 103,
 (515, 749): 103,
 (568, 745): 102,
 (356, 357): 100,
 (363, 511): 99,
 (551, 571): 98,
 (551, 809): 96,
 (502, 514): 95,
 (551, 794): 

In [7]:
max(courses_dict, key=courses_dict.get)

(551, 566)

### Составляем отчет 


Создадим функцию **recommend**:

In [8]:
def recommend(course):
    rec_list = []
    for pair in courses_dict.keys(): 
        if pair[0] == course: # проверка по первому значению в ключе (паре курсов)
            rec_list.append((pair[1])) # добавляем значение курса(-ов), который(-е) нашлись в паре с рассматриваемым курсом
        if pair[1] == course: # проверка по второму значению в ключе (паре курсов)
            rec_list.append((pair[0])) # добавляем значение курса(-ов), который(-е) нашлись в паре со вторым значением в ключе
        if len(rec_list) == 2:
            break
    return rec_list

Оформим **таблицу** с рекомендациями для продакт-менеджера и отдела маркетинга:

In [9]:
recomm_df = pd.DataFrame(columns = ['Курс для рекомендации № 1', 'Курс для рекомендации № 2'])
course_1 = None 
course_2 = None

for i in courses_dict: 
    reccomend = recommend(i[0]) # заводим переменную, которая проверит совпадения с первым значением в ключе словаря 
    if len(reccomend) == 2: # если количество парных с текущим курсом равно 2
        recomm_df.loc[i[0]] = [reccomend[0], reccomend[1]]
    elif len(reccomend) == 1: # если количество парных с текущим курсом равно 1
        recomm_df.loc[i[0]] = [reccomend[0], course_2]
    else:
        recomm_df.loc[i[0]] = [course_1, course_2]
    course_1 = None
    course_2 = None
    
for i in courses_dict: 
    reccomend = recommend(i[1]) # заводим переменную, которая проверит совпадения со вторым значением в ключе словаря
    if len(reccomend) == 2: # если количество парных с текущим курсом равно 2
        recomm_df.loc[i[1]] = [reccomend[0], reccomend[1]]
    elif len(reccomend) == 1: # если количество парных с текущим курсом равно 1
        recomm_df.loc[i[1]] = [reccomend[0], course_2]
    else:
        recomm_df.loc[i[1]] = [course_1, course_2]
    course_1 = None
    course_2 = None

Переименуем первый столбец:

In [10]:
recomm_df.index.name = 'Курс'
recomm_df

,Курс для рекомендации № 1,Курс для рекомендации № 2
Курс,,
551,566,515
515,551,489
489,551,515
523,551,515
566,551,794
...,...,...
1199,523,515
833,570,507
1201,1144,1186


Проверка:

In [11]:
i = 0
num = 1200
for k, v in courses_dict.items():
    if k[1] == num or k[0] == num:
        print(k, v)
        
        if i > 5:
            break
        i += 1

(1104, 1200) 1
(866, 1200) 1
(794, 1200) 1


Экспортируем датафрейм в Excel-файл:

In [12]:
recomm_df.to_excel(r'Recommendation_table4.xlsx')

Финальную таблицу в полном формате можно посмотреть [здесь](https://drive.google.com/file/d/1xc_5nW-J5NdDf6yJrU4G7sxWKfwAjHFI/view?usp=sharing). 

#### Комментарии по решению:

В представленной таблице приведены для каждого обучающего курса рекомендации по покупкам дополнительных курсов. Определение рекомендованных курсов осуществлено по следующей логике: 
    - проведен анализ совместных покупок для каждого курса с целью определить пары курсов, которые совершаются чаще всего в одной корзине с текущим курсом;
    - проверяются оба значения в паре на поиск рекомендуемых курсов для каждого значения, отбираем наиболее популярные пары в Рекомендацию №1 и №2.